In [1]:
from utils.tools import getPathList, getSEED4Data
import numpy as np
from collections import Counter
import numpy

In [2]:
data_path = r'C:\Users\Noman\Desktop\Github\CTL_Scratch\Trial\Data'

# Get list of .mat files
path_list = getPathList(data_path)

# Load and process the data
train_features, test_features, train_labels, test_labels = getSEED4Data(path_list)

In [3]:
# Convert lists to NumPy arrays for inspection
train_features = np.array(train_features)
test_features = np.array(test_features)

# Print shapes and label distributions
print('Training features shape:', train_features.shape)
print('Testing features shape:', test_features.shape)
print('Training labels length:', len(train_labels))
print('Testing labels length:', len(test_labels))
print('Training label distribution:', Counter(train_labels))
print('Testing label distribution:', Counter(test_labels))

Training features shape: (1735, 62, 5)
Testing features shape: (770, 62, 5)
Training labels length: 1735
Testing labels length: 770
Training label distribution: Counter({1: 530, 2: 455, 0: 381, 3: 369})
Testing label distribution: Counter({0: 297, 2: 160, 3: 160, 1: 153})


In [4]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import geoopt  # For Riemannian optimization
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from pyriemann.tangentspace import TangentSpace

def compute_spd_matrices(features):
    epsilon = 1e-5
    return np.array([f @ f.T + epsilon * np.eye(f.shape[0]) for f in features])

# Assume train_features and test_features are numpy arrays of shape (samples, channels, time)
train_covs = compute_spd_matrices(train_features)  # shape: (N_train, channels, channels)
test_covs = compute_spd_matrices(test_features)    # shape: (N_test, channels, channels)

# --------------------------------
# Step 2: Tangent Space Mapping
# --------------------------------
ts = TangentSpace(metric='riemann', tsupdate=False)
X_train_ts = ts.fit_transform(train_covs)  # shape: (N_train, feature_dim)
X_test_ts = ts.transform(test_covs)        # shape: (N_test, feature_dim)

# --------------------------------
# Step 3: Standardize Features
# --------------------------------
scaler = StandardScaler()
X_train_ts = scaler.fit_transform(X_train_ts)
X_test_ts = scaler.transform(X_test_ts)

# --------------------------------
# Step 4: Convert SPD matrices to torch tensor (for riemannian conv)
# --------------------------------
train_covs_tensor = torch.tensor(train_covs, dtype=torch.float32)
test_covs_tensor = torch.tensor(test_covs, dtype=torch.float32)

y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

# --------------------------------
# Step 5: Prepare DataLoader
# --------------------------------
train_dataset = TensorDataset(train_covs_tensor, y_train_tensor)
test_dataset = TensorDataset(test_covs_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\Noman\miniconda3\envs\mshcl\lib\site-packages\pyriemann\utils\mean.py:641: UserWarning: Convergence not reached
  warnings.warn("Convergence not reached")

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    ap

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from pyriemann.tangentspace import TangentSpace
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight


In [13]:
import numpy

In [16]:
class RiemannianConv(nn.Module):
    def __init__(self, in_channels):
        super(RiemannianConv, self).__init__()
        # Weight matrix to multiply SPD matrices (like a filter)
        # Initialized close to identity for stability
        self.weight = nn.Parameter(torch.eye(in_channels) + 0.01 * torch.randn(in_channels, in_channels))

    def forward(self, x):
        # x: (batch_size, channels, channels) SPD matrices
        # Perform W^T * X * W for each SPD matrix X in batch
        W = self.weight
        Wt = W.t()
        # batch multiplication: Wt @ X @ W
        out = torch.matmul(torch.matmul(Wt, x), W)
        return out

# --------------------------------
# Step 7: Define Model (RiemannianConv + Tangent Space + MLP)
# --------------------------------
class RiemannianModel(nn.Module):
    def __init__(self, channels, num_classes):
        super(RiemannianModel, self).__init__()
        self.riemannian_conv = RiemannianConv(channels)
        self.ts = TangentSpace(metric='riemann', tsupdate=False)  # from pyriemann
        # Calculate tangent space output dim
        self.tangent_dim = (channels * (channels + 1)) // 2
        self.mlp = nn.Sequential(
            nn.Linear(self.tangent_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x shape: (batch, channels, channels)
        x = self.riemannian_conv(x)  # SPD matrices remain SPD
        # Need to transform batch of SPD matrices to tangent space vectors
        # pyriemann TangentSpace expects numpy arrays, so convert back and forth:
        import numpy
        x_np = x.detach().cpu().numpy()
        x_ts = self.ts.transform(x_np)  # shape: (batch, tangent_dim)
        x_ts = torch.tensor(x_ts, dtype=torch.float32, device=x.device)
        out = self.mlp(x_ts)
        return out

# --------------------------------
# Step 8: Initialize model, loss, optimizer
# --------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_channels = train_covs.shape[1]
num_classes = len(np.unique(train_labels))

model = RiemannianModel(input_channels, num_classes).to(device)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [17]:
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# --------------------------------
# Step 10: Evaluation
# --------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.detach().cpu().tolist())
        all_labels.extend(y_batch.detach().cpu().tolist())

acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {acc * 100:.2f}%")
print("Classification Report:\n", classification_report(all_labels, all_preds))

RuntimeError: Numpy is not available

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from pyriemann.tangentspace import TangentSpace
from torch.utils.data import DataLoader, TensorDataset

# ----------------------------
# Step 1: Compute SPD Matrices
# ----------------------------
def compute_spd_matrices(features):
    epsilon = 1e-5
    return np.array([f @ f.T + epsilon * np.eye(f.shape[0]) for f in features])

train_covs = compute_spd_matrices(train_features)  # shape: (1735, 62, 62)
test_covs = compute_spd_matrices(test_features)    # shape: (770, 62, 62)

# ----------------------------
# Step 2: Tangent Space Mapping
# ----------------------------
ts = TangentSpace(metric='riemann', tsupdate=False)
X_train_ts = ts.fit_transform(train_covs)  # shape: (1735, 1953)
X_test_ts = ts.transform(test_covs)        # shape: (770, 1953)

# ----------------------------
# Step 3: Standardize Features
# ----------------------------
scaler = StandardScaler()
X_train_ts = scaler.fit_transform(X_train_ts)
X_test_ts = scaler.transform(X_test_ts)

# ----------------------------
# Step 4: Prepare Tensors
# ----------------------------
X_train_tensor = torch.tensor(X_train_ts, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_ts, dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32)

# ----------------------------
# Step 5: Define MLP Model
# ----------------------------
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

input_dim = X_train_ts.shape[1]
num_classes = len(np.unique(train_labels))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
model = MLP(input_dim, num_classes).to(device)
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Use in CrossEntropyLoss
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ----------------------------
# Step 6: Train the Model
# ----------------------------
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# ----------------------------
# Step 7: Evaluate the Model
# ----------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.detach().cpu().tolist())
        all_labels.extend(y_batch.detach().cpu().tolist())


acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {acc * 100:.2f}%")
print("Classification Report:\n", classification_report(all_labels, all_preds))


C:\Users\Noman\miniconda3\envs\mshcl\lib\site-packages\pyriemann\utils\mean.py:641: UserWarning: Convergence not reached
  warnings.warn("Convergence not reached")

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Noman\miniconda3\envs\mshcl\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    ap

Epoch 1/50, Loss: 1.3568
Epoch 2/50, Loss: 1.0050
Epoch 3/50, Loss: 0.5578
Epoch 4/50, Loss: 0.2739
Epoch 5/50, Loss: 0.1337
Epoch 6/50, Loss: 0.0763
Epoch 7/50, Loss: 0.0434
Epoch 8/50, Loss: 0.0362
Epoch 9/50, Loss: 0.0409
Epoch 10/50, Loss: 0.0274
Epoch 11/50, Loss: 0.0146
Epoch 12/50, Loss: 0.0193
Epoch 13/50, Loss: 0.0054
Epoch 14/50, Loss: 0.0120
Epoch 15/50, Loss: 0.0057
Epoch 16/50, Loss: 0.0056
Epoch 17/50, Loss: 0.0071
Epoch 18/50, Loss: 0.0095
Epoch 19/50, Loss: 0.0054
Epoch 20/50, Loss: 0.0044
Epoch 21/50, Loss: 0.0112
Epoch 22/50, Loss: 0.0221
Epoch 23/50, Loss: 0.0101
Epoch 24/50, Loss: 0.0026
Epoch 25/50, Loss: 0.0015
Epoch 26/50, Loss: 0.0011
Epoch 27/50, Loss: 0.0008
Epoch 28/50, Loss: 0.0005
Epoch 29/50, Loss: 0.0006
Epoch 30/50, Loss: 0.0019
Epoch 31/50, Loss: 0.0045
Epoch 32/50, Loss: 0.0307
Epoch 33/50, Loss: 0.0423
Epoch 34/50, Loss: 0.0232
Epoch 35/50, Loss: 0.0082
Epoch 36/50, Loss: 0.0008
Epoch 37/50, Loss: 0.0004
Epoch 38/50, Loss: 0.0004
Epoch 39/50, Loss: 0.